## CHAPTER #2 - HANDLING DATA 

## Importing dependencies 

In [ ]:
# Importing necessary packages 
%pip install pandas 
%pip install numpy
%pip install matplotlib
%pip install scikitlearn

In [ ]:
import numpy as np 
import matplotlib as mpl
import pandas as pd
from matplotlib import pyplot as plt

## FINANCE DATA 
The following datasets provide metrics partitioned by market capitalization, price, volatility, and turnover. The stock market activity metrics are partitioned by decile and the ETP metrics by quartile. 

I want to look into how cancellation rate (cancel to trade) is affected by stock volatility.  
I will look at different deciles  (the market capitilsation of 10 businesses) and the Market cap decile column as well as its Volatility to see how cancellation rate changes with volatility.   

**Hypothesis**- I would hypothesise that the greater the volaitilty the greater the rate of cancellation. 

In our data schema the following are defined:  
**Market Cap  Decile(n)** - what is the decile_cancel_to_trade (number of cancelled trades/ number of successful trades) for that capitalisation at that date. 
-  will be renamed to "Cancellation rate".    
                        

**Volatility Decile(n)** - the amount of statistical variation within each stock decile (e.g decile 9) as that date. 
- will be renamed to "Volatility"

## Can we predict the the cancelllation rate of a stock based on its volatility?
We will use linear regression to find out.

In [ ]:
decile_path= "/Users/admin/Desktop/Data Science Career /Python/Python Projects/Linear_Regression/data/decile_cancel_to_trade_stock.csv"
#Saving path name as variable for read csv argument 


In [ ]:
decile_to_cancel_raw=pd.read_csv(decile_path) #importing file as a pandas dataframe

In [ ]:
decile_to_cancel_raw.head()
decile_to_cancel_raw.tail()                   #insight into what our data looks like 

## EXPLORATORY DATA ANALYSIS AND DATA PRE-PROCESSING


I will utilise exploratory data analysis (EDA) to identify which decile has the most linear pattern in order to utilise my linear regression model.  
This is to isolate only volatility and cancellation features for one set of independent and dependent variables. 

### ASSUMPTIONS OF LINEAR REGRESSION
Let us investigate decile_1 to see if it is a good candidate to be modelled by linear regression. 
For this to be true, there must be:
1) Strong negative or positive correlation
2) Linearity in the data points
3) Strong homoscedacity (data points maintian similar deviation throughtout all values of the independent variable)
4) Normality of errors

#### Decile 1 and Linear Regression

In [ ]:
#Extracting features of market cap decile 1 
#independent variable = volatility decile1 (deviation in the stock prices  in this decile for each date)
#dependent varaible = market cap decile1 (the cancel to trade of stock each date for groups of 10 businesses in the lowest market capitalisation)
decile_1 = decile_to_cancel_raw[["Market Cap Decile1","Volatility Decile1"]] 

In [ ]:
decile_1.head() #what does our data look like

In [ ]:
decile_1 = decile_1.rename(columns={'Market Cap Decile1':'Cancellation rate','Volatility Decile1':'Volatility'})
#renaming columns since we know we are in decile 1 of the decile_cancel_to_trade file

## Does decile 1 fit our assumptions?
Let us see if our data for decile 1 fits our assumptions.    
To test this, I will make a basic plot of the two features.

### Linearity
Plot our indepdent variable vs depedent variable as a scatterplot 

In [ ]:
x_1= decile_1[['Cancellation rate']] #assigning columns to the variable x
y_1 = decile_1[['Volatility']]        #assigning columns to the variable y

In [ ]:
plot_linear = plt.scatter(y_1, x_1) #scatter plot of our two features for deile 1

From this plot we can see a few key details, namely: 
1) Our data has a few outliers.  
2) Our data does follow a linear relationship with most values condensed around the centre point of the volatility scale.  
3) The linearity is present but does not have a strong gradient meaning the volatility in decile 1 does not have much predictive power in regards to the cancellation rate. I will explore  different features for this model i.e different deciles. 

#### Decile 9 and Linear Regression

In [ ]:
#Extracting features of market cap decile 1 
decile_7 = decile_to_cancel_raw[["Market Cap Decile7","Volatility Decile7"]] 

In [ ]:
y_2 = decile_7[['Market Cap Decile7']] #assigning columns to the variable x
x_2 = decile_7[['Volatility Decile7']] #assigning columns to the variable x

In [ ]:
plot_linear = plt.scatter(x_2, y_2) #scatter plot of our two features for decile 9

From this plot we can see that:
1) There is a strong positive linear relationship, therefore volatililty does have predictive power for cancellation rates. 
2) There is heteroscedacity in the raw data, therefore we may need to apply some kind of transformation to the data but I will make the regression model and then check for homoscedacity in the residuals. 
3) Interetsing though since this high heteroscedcaity indicates that as stock get more volatile purchasing decisions become more extreme. 

In [ ]:
decile_7 = decile_7.rename(columns={'Market Cap Decile7':'Cancellation rate','Volatility Decile7':'Volatility'})
#renaming columns since we know we are in decile 9 of the cancel_to_trade file

In [ ]:
plot_box = plt.boxplot(x_2) #creating a boxplot of our independent variable

In [ ]:
plot_box = plt.boxplot(y_2) #creating a boxplot of our dependent variable

### Missing Values 

In [ ]:
missing_values = decile_7.isnull().sum()
print(missing_values)
#No missing values in the data 

From these plots we can see the presence of:
1) A significant number of outliers outside the maximum range of the dataset.
2) We will use the interquartile range method to impute these values.

## OUTLIER REMOVAL 
I will use the method of removing values above using upper and lower bounds based on quartiles. 

In [ ]:
#Step 1- compute Q1 and Q3
#Cancellation rates 
Q1C = decile_7['Cancellation rate'].quantile(0.25) #quartile 1 of cancellation rates 
Q3C = decile_7['Cancellation rate'].quantile(0.75) ##quartile 3 of cancellation rates 
print(Q1C)
print(Q3C)




In [ ]:
#Volatiltiy 
Q1V = decile_7['Volatility'].quantile(0.25)         #quartile 1 of volatility  
Q3V = decile_7['Volatility'].quantile(0.75)         #quartile 3 of volatility  
print(Q1V)
print(Q3V)

In [ ]:
#Step 2 - Compute IQR
#Cancellation rate 
IQRC = Q3C - Q1C
print(IQRC)

In [ ]:
#Volatility 
IQRV= Q3V -Q1V
print(IQRV)

In [ ]:
#Step 3 - Find the upper bound and lower bound
#Cancellation rate
upper_b_Canc = Q3C + 1.5*IQRC
print(upper_b_Canc)

In [ ]:
#Volatility 
upper_b_Vol = Q3V + 1.5*IQRV
print(upper_b_Vol)

In [ ]:
#Number of outliers on upper end 
#Cancellation rate
decile_7[decile_7['Cancellation rate'] > upper_b_Canc].count()


In [ ]:
#Volatility
decile_7[decile_7['Volatility'] > upper_b_Vol].count()
print((69/3329)*100) #percentage of values to impute. Is not excessive ≈ 2%

### WINSORISATION 
A method of dealing with systematic outliers to maintain distribution of data. 

Through our EDA, I learned that our outliers are only present in  the upper tail , truncating them would remove information from the data and make our model have 
lower predictive power at the high end.   
Therefore I will use Winsorisation to cap them  to the max of Q3 + 1.5*IQR - https://www.datacamp.com/tutorial/winsorized-mean

In [ ]:

#Cancellation rate 
decile_7['Cancellation rate'] = decile_7['Cancellation rate'].clip (upper = upper_b_Canc) #winsorising cancellation rate 
decile_7[decile_7['Cancellation rate'] > upper_b_Canc].count()  


In [ ]:
#Volatility
decile_7['Volatility'] = decile_7['Volatility'].clip (upper = upper_b_Vol)                 #winsorising volatility  
decile_7[decile_7['Volatility'] > upper_b_Vol].count()  

In [ ]:
# New  values of x and y with Winsoration 
y_3 = decile_7[['Cancellation rate']]
x_3 = decile_7[['Volatility']]

In [ ]:
plt.boxplot(x_3) #New boxplot of cancellation rate with Winsorisation

In [ ]:
plt.boxplot(y_3) #New boxplot of volatility with Winsorisation

In [ ]:
plt.hist(x_3)   #New histogram of Cancellation rate with Winsorisation

In [ ]:
plt.hist(y_3)   #New boxplot of Volatility with Winsorisation

Now that we have the pre-processed data with linearity and outliers handled we can  use our linear regression model.

### LINEAR REGRESSION MODELLING 

In [ ]:
import sys
sys.path.append('/Users/admin/Desktop/Data Science Career /Python/Python Projects/Linear_Regression')
from model import LinearRegression


In [ ]:
finance_model = LinearRegression(0.0001, 10000)

In [ ]:
#Converting data into pandas series 
x = decile_9['Cancellation rate']
y = decile_9['Volatility']

In [ ]:
finance_model.vectorise_x(x)

In [ ]:
finance_model.vectorise_y(y)

In [ ]:
finance_model.optimise()

In [ ]:
finance_model.residuals()

In [ ]:
finance_model.plot_residuals()

In [ ]:
finance_model.rsquared()

In [ ]:
finance_model.mse()

In [ ]:
%pip install scikit-learn 

In [ ]:
from sk import LinearRegression as lr